## import packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd # for creation of data frames
import numpy as np #
import pyls # PLS: behavioral and mean-centered
import seaborn as sns
import nibabel as nib # for loading niftis etc. as arrays
import matplotlib.pyplot as plt # for plotting data
#the output of plotting commands is displayed inline, directly below the code cell that produced it
%matplotlib inline 
import os,glob
from nilearn import plotting, input_data, image #for plotting & working with niftis

import pathlib

###################### update path!! #####################################
base_path = '/root_dir/' ####
base_path = '/home/tumnic/sepp/neuroenergeticslab_notebooks/erc-wp1/manuscript/' ####
##########################################################################

#import qBOLD functions
#import sys
#sys.path.append(os.path.join(base_path + 'scripts/'))
#import mqBOLD_functions as qB


### define FSL directories ######################################################
os.environ["FSLDIR"]='/usr/share/fsl/5.0'
os.environ["FSLOUTPUTTYPE"]='NIFTI_GZ'
os.environ["FSLTCLSH"]='/usr/bin/tclsh'
os.environ["FSLWISH"]='/usr/bin/wish'
os.environ["FSLMULTIFILEQUIT"]="True"
os.environ["LD_LIBRARY_PATH"]='/usr/share/fsl/5.0:/usr/lib/fsl/5.0'
#################################################################################

## define variables (please check!)

In [ ]:
data_dir = base_path + 'data/'
derivatives_dir = data_dir + 'derivatives'
raw_dir = data_dir + 'rawdata'
results_dir = base_path + 'results'
MNI_2mm_brain = derivatives_dir + '/MNI152_T1_2mm_brain.nii.gz'

sns.set_style("whitegrid")

coords=(-15, 0, 15, 30, 45, 60, 75, 90)

## run qBOLD toolbox
%run qBOLD_fun.ipynb

sns.set_style("whitegrid")
fontsize=20
plt.rcParams['legend.title_fontsize'] = 'x-large'

In [ ]:
## Please which CMRO2 version to use ##
###################################################

#CMRO2_mode = 'orig' ## orig = CMRO2 maps NOT corrected for CBV changes, only CMRO2 maps of subj > 55 in CALC condition are corrected for CBV increases
CMRO2_mode = 'corrected' ## CMRO2 CALC maps corrected for CBV changes (but MEM maps are not corrected)

##### Choose if semi-quantitative CMRO2 or fully quantitative CMRO2!
################################################################################
CMRO2_quant = 'semi-quant'
#CMRO2_quant = 'fully-quant'

## Please specify task & baseline ##
####################################

#baseline='rest'
baseline='control'
task='calc'
#task='mem'

conds = [baseline, task]
contrast=task+baseline

## Please specify subject list##
################################

if baseline == 'control' and task == 'calc':
    #N=40
    sids = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68]
    #sids = [58, 59, 60, 61, 63, 64, 65, 66, 67, 68]
    #N=10
    sids_10 = [58, 59, 60, 61, 63, 64, 65, 66, 67, 68]
else: #N=30, without p053
    sids = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55]


##########################################################
## please select which group task mask to apply!! ##
##########################################################

#masks = ['Yeo', 'PLS BOLD', 'PLS CBF', 'GLM BOLD']
#masks = ['PLS CBF', 'PLS CMRO2', 'PLS OEF']
mask = [ 'PLS BOLD'] ## this is the PLS BOLD group mask
#mask = ['PLS CMRO2']
#masks = ['GLM_BOLD']


### other variables ##############
##################################

N_subj = str(len(sids))
GM_thresh = 0.5
BSR_thr=2

################### Please specify if only GM mask or GM + CBV + R2' mask!! ######################################

#whole_brain_masking = 'GMmask'
whole_brain_masking = 'CBV_R2smask'

############## Please specify if MNI 2mm space or lowres MNI space !! #####################
resolution = 'res-2'
#resolution = 'lowres' ##here, it's also MNI 2mm space, but CMRO2 was calculated beforehands in lowres native space!

MNI_2mm_brain = derivatives_dir + '/MNI152_T1_2mm_brain.nii.gz'

if whole_brain_masking == 'GMmask':
    whole_brain_mask = os.path.join( derivatives_dir, 'task-all_space-MNI152_res-2_SNR_YEO_group_mask.nii.gz')
if whole_brain_masking == 'CBV_R2smask':
    whole_brain_mask = os.path.join(derivatives_dir, 'N40_CBV_R2s_mask_'+resolution+'.nii.gz')
    #whole_brain_mask = os.path.join(derivatives_dir, 'N' + str(len(sids)) + '_CBV_R2s_mask.nii.gz')
       
##########################################################################################

masker_GM = input_data.NiftiMasker(mask_img=whole_brain_mask)
mask_img = masker_GM.fit_transform(MNI_2mm_brain)



### Blue-red color palette

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl

colors = ["darkblue", "blue", "dodgerblue",  "whitesmoke","darkorange", "red", "darkred"]
BlueRed = LinearSegmentedColormap.from_list("mycmap", colors)

fig, ax = plt.subplots(figsize=(3,0.5))
fig.subplots_adjust(bottom=0.5)

bounds = [-6, -4, -2, 0, 2, 4, 6]
norm = mpl.colors.Normalize(vmin=-6, vmax=6)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=BlueRed), 
             cax=ax, ticks=bounds, orientation='horizontal')

## inverted colormap: BlueRed_r
colors = [ "darkred","red", "darkorange", "whitesmoke", "dodgerblue", "blue","darkblue"]
BlueRed_r = LinearSegmentedColormap.from_list("mycmap", colors)

fig, ax = plt.subplots(figsize=(8, 2))
fig.subplots_adjust(bottom=0.5)

bounds = [-6, -4, -2, 0, 2, 4, 6]
norm = mpl.colors.Normalize(vmin=-6, vmax=6)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=BlueRed_r), 
             cax=ax, ticks=bounds, orientation='horizontal')

## Fig 3A: draw Davis model

In [ ]:
sns.set_style("whitegrid")
#a = 0.58; b=1.5; M=9.8 ## see paper of Lin & Fox, 2008
#a = 0.14; b=0.91; M=14.9 ## see paper of Griffeth & Buxton, 2011
#a = -0.05; b=0.98; M=10 ## see paper of Gagnon & Boas, 2016
a = 0.23; b=1.3; M=5.5 ## Chen & Pike, 2009 for alpha, b in Bright et al, 2019, M from Lajoie et al., 2016 

# Range of x and y axis
CMRO2 = np.arange(-0.5, 0.5, 0.01)
CBF = np.arange(-0.5, 0.5, 0.01)
  
# Creating 2-D grid of features
[X, Y] = np.meshgrid(CMRO2, CBF)
  

# Davis model
Z = M*(1-((Y+1)**(a-b))*(X+1)**b)

CMRO2_perc = np.arange(-50, 50, 1)
CBF_perc = np.arange(-50, 50, 1)
[X, Y] = np.meshgrid(CMRO2_perc, CBF_perc)


fig, ax = plt.subplots(1, 1, figsize=(7,5))
#scatter = ax.scatter(X, Y, c=Z, s=2, cmap = BlueRed, vmin=-3, vmax=3)

#cbar = plt.colorbar(CS)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.hlines(0, -45, 45, color='black', alpha=0.3, linestyles='dashed')
plt.vlines(0, -45, 45, color='red',alpha=1, linestyles='dashed')

#ax.clabel(CS, CS.levels, inline=True,fontsize=10)

#ax.set_title('BOLD predicted by Davis Model')
ax.set_xlabel('ΔCMRO2 [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)
ax.set_xlim(-45,45)
ax.set_ylim(-45,45)


# cmap gives the color scheme, and levels define that 6 contour lines will be drawn for BOLD value between -0.2 and 0.2
CS = ax.contour(X, Y, Z, cmap = BlueRed, levels=np.linspace(-3,3,8))
# format in percentage
def fmt(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s} \%" if plt.rcParams["text.usetex"] else f"{s} %"
ax.clabel(CS, CS.levels, inline=True,fontsize=12, fmt=fmt)

#cbar = plt.colorbar(CS)
cbar = plt.colorbar(CS, ticks=CS.levels)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_yticklabels([str(round(level, 2)) for level in CS.levels]) 


for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

fig.savefig(results_dir + '/Davis.model.png', dpi=300, bbox_inches='tight')
plt.show()

## plot dependence of CMRO2 on CBV and CBF

In [ ]:
sns.set_style("whitegrid")

  
Hct = 40 
O2sat=98
C = 4/3*267.61918*np.pi*0.264*(Hct/100)*0.85*3; # 317 Hz
CaO2 = 0.334 * float(Hct) * 55.6 * float(O2sat)/100;


############################################
## baseline CMRO2 ######
############################################
R2prime_control = 5.3

## vary across values of CBV and CBF
CBV = np.arange(4,6,0.01)
CBF = np.arange(40, 60, 0.1)
[X, Y] = np.meshgrid(CBV, CBF)
CMRO2_control = R2prime_control / (C*X) * Y * CaO2


fig, ax = plt.subplots(1, 1, figsize=(7,5))

#cbar = plt.colorbar(CS)
#cbar.set_label("CMRO2", fontsize=16)
#cbar.ax.tick_params(labelsize=14)

#plt.hlines(0, -45, 45, color='black', alpha=0.3, linestyles='dashed')
#plt.vlines(0, -45, 45, color='red',alpha=1, linestyles='dashed')

#ax.clabel(CS, CS.levels, inline=True,fontsize=10)

#ax.set_title('BOLD predicted by Davis Model')
ax.set_xlabel('CBV', fontsize=fontsize)
ax.set_ylabel('CBF', fontsize=fontsize)
#ax.set_xlim(-45,45)
#ax.set_ylim(-45,45)


# cmap gives the color scheme, and levels define that 6 contour lines will be drawn for BOLD value between -0.2 and 0.2
CS = ax.contour(X, Y, CMRO2_control, cmap = BlueRed, levels=np.linspace(80,200,10))
# format in percentage
def fmt(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s}" if plt.rcParams["text.usetex"] else f"{s} %"
ax.clabel(CS, CS.levels, inline=True,fontsize=12, fmt=fmt)

#cbar = plt.colorbar(CS)
cbar = plt.colorbar(CS, ticks=CS.levels)
cbar.set_label("CMRO2", fontsize=16)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_yticklabels([str(round(level, 2)) for level in CS.levels]) 


for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

plt.show()



## change in R2' fixed
R2prime_calc = 5.4
R2prime_control = 5.3
R2prime_perc = (R2prime_calc-R2prime_control)/R2prime_control *100

############################################
## task changes; CBV scaling is variable ######
############################################

## change in CBF fixed
CBF_calc = 50.5
CBF_control = 46.5
CBF_perc = (CBF_calc - CBF_control)/CBF_control*100

### change in variable
CBV_calc_orig = 4.9
CBV_control_orig = 4.6
CBV_perc = (CBV_calc_orig - CBV_control_orig) / CBV_control_orig * 100

percent_values = [1, 1.25, 1.666667, 2.5]
CMRO2_delta_arr = np.zeros((len(percent_values),1))
CMRO2_perc_arr = np.zeros((len(percent_values),1))
CMRO2_calc_arr = np.zeros((len(percent_values),1))
CMRO2_control_arr = np.zeros((len(percent_values),1))

for p, perc in enumerate(percent_values):
    CBV_calc = CBV_calc_orig/perc
    CBV_control = CBV_control_orig/perc
    CBV_perc = (CBV_calc - CBV_control) / CBV_control * 100


    ## calculate OEF and CMRO2 via Fick's principle 
    CMRO2_calc = R2prime_calc / (C*CBV_calc) * CBF_calc * CaO2
    CMRO2_control = R2prime_control / (C*CBV_control) * CBF_control * CaO2
    CMRO2_delta = CMRO2_calc - CMRO2_control
    CMRO2_perc = (CMRO2_calc - CMRO2_control)/CMRO2_control * 100
    CMRO2_calc_arr[p] = CMRO2_calc
    CMRO2_control_arr[p] = CMRO2_control
    CMRO2_delta_arr[p] = CMRO2_delta
    CMRO2_perc_arr[p] = CMRO2_perc
    

CBV_scaling = ['2.5%', '2%', '1.5%', '1%']


## plot absolute CMRO2 values
fig, ax = plt.subplots(1, 1, figsize=(7,5))

ax.set_xlabel('CBV WM normalization', fontsize=fontsize)
ax.set_ylabel('CMRO2', fontsize=fontsize)

ax.scatter(CBV_scaling, CMRO2_calc_arr, marker='+', color='red', s=100, label='CALC')
ax.scatter(CBV_scaling, CMRO2_control_arr, marker='+', color='black', s=100, label='CTRL')

ax.legend(fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

plt.show()


## plot delta and percent values
fig, ax = plt.subplots(1, 1, figsize=(7,5))

ax.set_xlabel('CBV WM normalization', fontsize=fontsize)
ax.set_ylabel('CMRO2', fontsize=fontsize)

ax.scatter(CBV_scaling, CMRO2_delta_arr, marker='*', color='black', s=100, label='ΔCMRO2')
ax.scatter(CBV_scaling, CMRO2_perc_arr, marker='+', color='black', s=100, label='%CMRO2')
ax.set_ylim(0, 14)

ax.legend(fontsize=fontsize)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

plt.show()

############################################
## task changes; CBF scaling is variable ######
############################################

## change in CBV is variable 
CBF_calc_orig = 50.5
CBF_control_orig = 46.5
CBF_perc = (CBF_calc_orig - CBF_control_orig)/CBF_control_orig*100

### change is fixed 
CBV_calc = 4.9
CBV_control = 4.6
CBV_perc = (CBV_calc - CBV_control) / CBV_control * 100


percent_values = [1, 1.25, 1.5, 1.75]
CMRO2_delta_arr = np.zeros((len(percent_values),1))
CMRO2_perc_arr = np.zeros((len(percent_values),1))
CMRO2_calc_arr = np.zeros((len(percent_values),1))
CMRO2_control_arr = np.zeros((len(percent_values),1))

for p, perc in enumerate(percent_values):
    CBF_calc = CBF_calc_orig*perc
    CBF_control = CBF_control_orig*perc
    CBF_perc = (CBF_calc - CBF_control) / CBF_control * 100

    ## calculate OEF and CMRO2 via Fick's principle 
    CMRO2_calc = R2prime_calc / (C*CBV_calc) * CBF_calc * CaO2
    CMRO2_control = R2prime_control / (C*CBV_control) * CBF_control * CaO2
    CMRO2_delta = CMRO2_calc - CMRO2_control
    CMRO2_perc = (CMRO2_calc - CMRO2_control)/CMRO2_control * 100
    CMRO2_calc_arr[p] = CMRO2_calc
    CMRO2_control_arr[p] = CMRO2_control
    CMRO2_delta_arr[p] = CMRO2_delta
    CMRO2_perc_arr[p] = CMRO2_perc
    

CBF_scaling = ['0%', '25%', '50%', '75%']


## plot absolute CMRO2 values
fig, ax = plt.subplots(1, 1, figsize=(7,5))

ax.set_xlabel('CBF upscaling', fontsize=fontsize)
ax.set_ylabel('CMRO2', fontsize=fontsize)

ax.scatter(CBF_scaling, CMRO2_calc_arr, marker='+', color='red', s=100, label='CALC')
ax.scatter(CBF_scaling, CMRO2_control_arr, marker='+', color='black', s=100, label='CTRL')

ax.legend(fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

plt.show()


## plot delta and percent values
fig, ax = plt.subplots(1, 1, figsize=(7,5))

ax.set_xlabel('CBF upscaling', fontsize=fontsize)
ax.set_ylabel('CMRO2', fontsize=fontsize)

ax.scatter(CBF_scaling, CMRO2_delta_arr, marker='*', color='black', s=100, label='ΔCMRO2')
ax.scatter(CBF_scaling, CMRO2_perc_arr, marker='+', color='black', s=100, label='%CMRO2')

ax.legend(fontsize=fontsize)
ax.set_ylim(0, 9)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

plt.show()

## Fig. 3B

In [ ]:
from nilearn import glm

if mask[0] == 'PLS BOLD':
        
    if whole_brain_masking == 'GMmask':
        BSR_masked_BOLD_YEO = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz'
        PLS_mask = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'_bin.nii.gz'

        BSR_cluster_thr = glm.threshold_stats_img(BSR_masked_BOLD_YEO, mask_img=None, alpha=0.05, threshold=2, height_control=None, cluster_threshold=30, two_sided=True)
        nib.save(BSR_cluster_thr[0],results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz')
        BSR_cluster_thr_nii = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz' 
    
    if whole_brain_masking == 'CBV_R2smask':
        whole_brain_mask = os.path.join(derivatives_dir, 'N40_CBV_R2s_mask_'+resolution+'.nii.gz')
        masker_GM = input_data.NiftiMasker(mask_img=whole_brain_mask)    
        BSR_masked = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_GMmask.nii.gz' 
        BSR_cluster_thr_nii = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz' 
        PLS_mask = results_dir + '/N' + N_subj +'_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'_bin.nii.gz'
        ! fslmaths {BSR_masked} -mas {whole_brain_mask} {BSR_cluster_thr_nii}
    


if mask[0] == 'PLS CBF':
    BSR_masked_BOLD_YEO = results_dir + '/N40_BSR_meanPLS_cbf_clusterBSR'+ resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz'
    PLS_mask = results_dir + '/N40_BSR_meanPLS_cbf_' +contrast+ '_' + whole_brain_masking + '_bin.nii.gz'
    BSR_cluster_thr = glm.threshold_stats_img(BSR_masked_BOLD_YEO, mask_img=None, alpha=0.05, threshold=2, height_control=None, cluster_threshold=30, two_sided=True)
    nib.save(BSR_cluster_thr[0],results_dir + '/N' + N_subj +'_BSR_meanPLS_cbf_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz')
    BSR_cluster_thr_nii = results_dir + '/N' + N_subj +'_BSR_meanPLS_cbf_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz' 

if mask[0] == 'PLS CMRO2':
    BSR_masked_BOLD_YEO = results_dir + '/N40_BSR_meanPLS_cmro2_clusterBSR'+ resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz'
    PLS_mask = results_dir + '/N40_BSR_meanPLS_cmro2_' +contrast+ '_' + whole_brain_masking + '_bin.nii.gz'
    BSR_cluster_thr = glm.threshold_stats_img(BSR_masked_BOLD_YEO, mask_img=None, alpha=0.05, threshold=2, height_control=None, cluster_threshold=30, two_sided=True)
    nib.save(BSR_cluster_thr[0],results_dir + '/N' + N_subj +'_BSR_meanPLS_cmro2_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz')
    BSR_cluster_thr_nii = results_dir + '/N' + N_subj +'_BSR_meanPLS_cmro2_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz' 



plot=plotting.plot_img(BSR_cluster_thr_nii, bg_img=MNI_2mm_brain, display_mode='z', threshold=0, cmap=BlueRed, colorbar = True, black_bg=False,
                    cut_coords = coords, title='N = ' + N_subj + ': mask', vmin = -7, vmax = 7)

## create mask that includes both posiitve and negative BOLD areas
! fslmaths {BSR_cluster_thr_nii} -abs -thr {BSR_thr} -bin {PLS_mask}
masker = input_data.NiftiMasker(mask_img = PLS_mask)
mask_arr = np.array(nib.load(PLS_mask).dataobj)
mask_len = len (mask_arr[abs(mask_arr)>0])
    
plot=plotting.plot_img(BSR_cluster_thr_nii, bg_img=MNI_2mm_brain, threshold = 2, display_mode='z', cmap=BlueRed, colorbar=True,
                cut_coords = coords,  title=   ' percchange',vmin=-7, vmax=7)
plot=plotting.plot_img(PLS_mask, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap='jet', colorbar=True,
                cut_coords = coords,  title=   ' mask',vmin=-7, vmax=7)

In [ ]:
#if mask[0] == 'PLS BOLD':
#    BSR_cluster_thr_nii = results_dir + '/N40_BSR_meanPLS_func_clusterBSR_' + resolution+'_' +contrast+'_' + whole_brain_masking +'.nii.gz'
#    if resolution != 'lowres':
#        BSR_cluster_thr_nii = results_dir + '/N40_BSR_meanPLS_func_' +contrast+ '_' + whole_brain_masking +'_YEOmask.nii.gz'
#        PLS_mask = results_dir + '/N40_BSR_meanPLS_func_' +contrast+ '_' + whole_brain_masking + '_bin.nii.gz'


## create mask that includes both posiitve and negative BOLD areas
#! fslmaths {BSR_cluster_thr_nii} -abs -bin {PLS_mask}
#masker = input_data.NiftiMasker(mask_img = PLS_mask)
#mask_arr = np.array(nib.load(PLS_mask).dataobj)
#mask_len = len (mask_arr[abs(mask_arr)>0])
    
#plot=plotting.plot_img(BSR_cluster_thr_nii, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=BlueRed, colorbar=True,
#                cut_coords = coords,  title=   ' mask',vmin=-7, vmax=7)

BOLD_percchange = np.zeros((len(sids), mask_len))
CBF_percchange = np.zeros((len(sids), mask_len))
CMRO2_percchange = np.zeros((len(sids), mask_len))
CBV_percchange = np.zeros((10, mask_len))

## SNR-GM mask (same mask was used for the PLS input)
SNR_GM_BOLD_mask_arr = np.array(nib.load(whole_brain_mask).dataobj)
SNR_GM_mask_len = len (SNR_GM_BOLD_mask_arr[abs(SNR_GM_BOLD_mask_arr)>0])
BOLD_percchange_GM = np.zeros((len(sids), SNR_GM_mask_len))
CBF_percchange_GM = np.zeros((len(sids), SNR_GM_mask_len))
CMRO2_percchange_GM = np.zeros((len(sids), SNR_GM_mask_len))
CBV_percchange_GM = np.zeros((10, SNR_GM_mask_len))

for i, ID in enumerate(sids): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    sub_dir = os.path.join(data_dir, sub)

    dir_anat = os.path.join(sub_dir, 'anat')
    dir_func = os.path.join(sub_dir, 'func')
    dir_perf = os.path.join(sub_dir, 'perf')

    dir_anat_deriv = os.path.join(derivatives_dir, sub, 'anat')
    dir_func_deriv = os.path.join(derivatives_dir, sub, 'func')
    dir_perf_deriv = os.path.join(derivatives_dir, sub, 'perf')
    dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')

    #load percent signal change image
    if resolution != 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-MNI152_desc-fmriprep_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 
        BOLD_percchange_GM[i, :] = masker_GM.fit_transform(BOLD_percchange_nii) 
    if resolution == 'lowres':
        BOLD_percchange_nii = nib.load(dir_func_deriv + '/' + sub + '_task-'+task + baseline+'_space-MNI152-lowres_desc-fmriprep_BOLD_percchange.nii.gz')
        BOLD_percchange[i, :] = masker.fit_transform(BOLD_percchange_nii) 
        BOLD_percchange_GM[i, :] = masker_GM.fit_transform(BOLD_percchange_nii) 
        
    ## for CBF, create %change    
    if resolution != 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbf.nii.gz'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbf.nii.gz'))/0.75
    if resolution == 'lowres':
        CBF_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbf.nii.gz'))/0.75
        CBF_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbf.nii.gz'))/0.75        
    CBF_percchange[i, :]  = (CBF_task - CBF_baseline) / CBF_baseline *100
    
    if resolution != 'lowres':
        CBF_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbf.nii.gz'))/0.75
        CBF_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbf.nii.gz'))/0.75
    if resolution == 'lowres':
        CBF_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbf.nii.gz'))/0.75
        CBF_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbf.nii.gz'))/0.75
    CBF_percchange_GM[i, :]  = (CBF_task_GM - CBF_baseline_GM) / CBF_baseline_GM *100
    
    ## for CMR02, create %change
    ## baseline
    if resolution != 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_desc-orig_cmro2.nii.gz'))
    if resolution == 'lowres':
        CMRO2_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))
    
    if CMRO2_quant == 'fully-quant':
        ## task
        if CMRO2_mode == 'corrected' and task == 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-CBV_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-CBV_cmro2.nii.gz'))
        if CMRO2_mode == 'orig' or task != 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-orig_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task =  masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))
                
    if CMRO2_quant == 'semi-quant':
        ## task
        if CMRO2_mode == 'orig' or task !='calc': ## without CBV correction
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant_cmro2.nii.gz'))
        if CMRO2_mode == 'corrected' and task == 'calc':  
            if resolution != 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant-corrected_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant-corrected_cmro2.nii.gz'))
    
    CMRO2_percchange[i, :]  = (CMRO2_task- CMRO2_baseline) / CMRO2_baseline *100 
    CMRO2_percchange[i, :][np.isinf(CMRO2_percchange[i, :])]  = np.nan
    

   
    ## GM mask
    ## baseline
    if resolution != 'lowres':
        CMRO2_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_desc-orig_cmro2.nii.gz'))
    if resolution == 'lowres':
        CMRO2_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))
    
    if CMRO2_quant == 'fully-quant':
        ## task
        if CMRO2_mode == 'corrected' and task == 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-CBV_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-CBV_cmro2.nii.gz'))
        if CMRO2_mode == 'orig' or task != 'calc': ## here we acquired 2 CBV, so these are the original data
            CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_desc-orig_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_desc-orig_cmro2.nii.gz'))
                
    if CMRO2_quant == 'semi-quant':
        ## task
        if CMRO2_mode == 'orig' or task !='calc': ## without CBV correction
            if resolution != 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant_cmro2.nii.gz'))
        if CMRO2_mode == 'corrected' and task == 'calc':  
            if resolution != 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152_desc-semi-quant-corrected_cmro2.nii.gz'))
            if resolution == 'lowres':
                CMRO2_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-'+task+'_base-'+baseline+'_space-MNI152-lowres_desc-semi-quant-corrected_cmro2.nii.gz'))                                 
                                                                                      
    CMRO2_percchange_GM[i, :]  = (CMRO2_task_GM - CMRO2_baseline_GM) / CMRO2_baseline_GM *100       
    CMRO2_percchange_GM[i, :][np.isinf(CMRO2_percchange_GM[i, :])]  = np.nan
    
    ## create image of CMRO2 percchange per subject ()
    #CMRO2_percchange_nii = masker_GM.inverse_transform(CMRO2_percchange_GM[i, :])
    #plot=plotting.plot_img(CMRO2_percchange_nii, bg_img=MNI_2mm_brain, display_mode='z', threshold=0, cmap=BlueRed, colorbar = True, black_bg=False,
    #                cut_coords = coords, title= sub + ': CMRO2 percchange', vmin = -50, vmax = 50)
    

## save % change
BOLD_percchange_median = np.nanmedian(BOLD_percchange, axis=0)
CBF_percchange_median = np.nanmedian(CBF_percchange, axis=0)
CMRO2_percchange_median = np.nanmedian(CMRO2_percchange, axis=0)

BOLD_percchange_median_GM = np.nanmedian(BOLD_percchange_GM, axis=0)
CBF_percchange_median_GM = np.nanmedian(CBF_percchange_GM, axis=0)
CMRO2_percchange_median_GM = np.nanmedian(CMRO2_percchange_GM, axis=0)

### plot all positive & negative voxels in one scatterplot

#### CMRO2 vs. CBF (BOLD als color)

In [ ]:
from matplotlib.colors import ListedColormap

colors_lightorange = ListedColormap(["navajowhite"])
colors_lightblue = ListedColormap(["lightskyblue"])

coords = (-30, -15, 0, 15, 30, 45, 60)
#DMN = os.path.join(tmp_dir,'Yeo2011_17Networks_MNI152_FreeSurferConformed2mm_MNI_DMN.nii.gz')
colors=BlueRed


BOLD_nifti = masker.inverse_transform(BOLD_percchange_median)
plot=plotting.plot_img(BOLD_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors, colorbar=True,
                cut_coords = coords,  title=   ' mask: BOLD increase / decrease',vmin=-1, vmax=1 )
plot=plotting.plot_img(BOLD_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors, colorbar=True,
                cut_coords = coords,  title=  ' mask: BOLD',vmin=-1, vmax=1,
                output_file=results_dir + '/BOLD_' + contrast +  '.pdf')

## plot %signal change in CMRO2 and CBF vs. BOLD (in color) ##
##############################################################

## group mask ##
###################

## plot in brain
overlap_CMRO2_CBF = np.zeros((3, len(CMRO2_percchange_median)))
overlap_CMRO2_CBF[0, CMRO2_percchange_median>0.01] = 1
overlap_CMRO2_CBF[0, CMRO2_percchange_median<-0.01] = -1
overlap_CMRO2_CBF[1, CBF_percchange_median>0.01] = 1
overlap_CMRO2_CBF[1, CBF_percchange_median<-0.01] = -1
overlap_CMRO2_CBF_sum = np.sum(overlap_CMRO2_CBF, axis=0)

CBF_CMRO2_nifti = masker.inverse_transform(overlap_CMRO2_CBF_sum)
plot=plotting.plot_img(CBF_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors, colorbar=True,
                cut_coords = coords,  title=  ' mask: CBF and CMRO2 increase / decrease', vmin=-1, vmax=1)
#plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)

##plot with BOLD
overlap_CMRO2_CBF[2, BOLD_percchange_median>0.001] = 1
overlap_CMRO2_CBF[2, BOLD_percchange_median<-0.001] = -1
overlap_CMRO2_CBF_sum = np.sum(overlap_CMRO2_CBF, axis=0)
CBF_CMRO2_nifti = masker.inverse_transform(overlap_CMRO2_CBF_sum)
plot=plotting.plot_img(CBF_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 1, display_mode='z', cmap=colors, colorbar=True,
                cut_coords = coords,  title=  ' mask: BOLD, CBF and CMRO2 increase / decrease')
#plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)


### get regression equation and plot line
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
#b, a = np.polyfit(CMRO2_percchange_median, CBF_percchange_median, deg=1)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(-30, 40, num=200)

## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(CMRO2_percchange_median, CBF_percchange_median, c=BOLD_percchange_median, s=2, cmap = colors, vmin=-1, vmax=1)

cbar = plt.colorbar(scatter)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)
# Plot regression line
#ax.plot(xseq, a + b * xseq, color="grey", lw=2.5, linestyle='dashed');
#ax.text(20, 40, 'b=' + str(round(b, 2)), fontsize=fontsize)
    

plt.hlines(0, -30, 40, color='grey', alpha=0.4, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')

ax.set_xlabel('ΔCMRO2 [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
    
ax.set_xlim(-30,40)
ax.set_ylim(-30,40)

fig.savefig(results_dir + '/CMRO2_CBF'+ '_mask_' +  contrast +  '_'+CMRO2_mode +'.png', dpi=300, bbox_inches='tight')

plt.show()


## GM SNR mask ##
#################

## plot only BOLD
BOLD = np.zeros((1, len(BOLD_percchange_median_GM)))
BOLD[0, BOLD_percchange_median_GM>0.001] = 1
BOLD[0, BOLD_percchange_median_GM<-0.001] = -1
BOLD_sum = np.sum(BOLD, axis=0)

BOLD_nifti = masker_GM.inverse_transform(BOLD_sum)

plot=plotting.plot_img(BOLD_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap='bwr', colorbar=True,
                cut_coords = coords,  title= 'SNR GM mask: BOLD increase / decrease')
#plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)

### get regression equation and plot line
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
#b, a = np.polyfit(CMRO2_percchange_median_GM, CBF_percchange_median_GM, deg=1)

# Create sequence of 100 numbers from 0 to 100 
#xseq = np.linspace(-30, 40, num=200)

## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(CMRO2_percchange_median_GM, CBF_percchange_median_GM, c=BOLD_percchange_median_GM, s=2, cmap = colors, vmin=-1, vmax=1)
# Plot regression line
#ax.plot(xseq, a + b * xseq, color="grey", lw=2.5, linestyle='dashed');
#ax.text(20, 40, 'b=' + str(round(b, 2)), fontsize=fontsize)

cbar = plt.colorbar(scatter)
cbar.set_label("ΔBOLD [%]", fontsize=16)
cbar.ax.tick_params(labelsize=14)

#ax.set_title("change in BOLD, median across subjects, SNR GM mask")

plt.hlines(0, -30, 40, color='grey', alpha=0.4, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')

ax.set_xlabel('ΔCMRO2 [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
ax.plot([-30, 40], [-30, 40], color='gray', linestyle='--')


ax.set_xlim(-30,40)
ax.set_ylim(-30,40)

fig.savefig(results_dir + '/CMRO2_CBF'+ '_mask_' + contrast +  '_'+CMRO2_mode +'_'+whole_brain_masking+'.png', dpi=300, bbox_inches='tight')

plt.show()

## plot in brain
overlap_CMRO2_CBF = np.zeros((3, len(CMRO2_percchange_median_GM)))
overlap_CMRO2_CBF[0, CMRO2_percchange_median_GM>0.01] = 1
overlap_CMRO2_CBF[0, CMRO2_percchange_median_GM<-0.01] = -1
overlap_CMRO2_CBF[1, CBF_percchange_median_GM>0.01] = 1
overlap_CMRO2_CBF[1, CBF_percchange_median_GM<-0.01] = -1
overlap_CMRO2_CBF_sum = np.sum(overlap_CMRO2_CBF, axis=0)

CBF_CMRO2_nifti = masker_GM.inverse_transform(overlap_CMRO2_CBF_sum)
plot=plotting.plot_img(CBF_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 1, display_mode='z', cmap=colors, colorbar=True,
                cut_coords = coords,  title= 'SNR GM mask: CBF and CMRO2 increase / decrease')
#plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)


#### NOT USED: Plot BOLD - CBF and CBV - BOLD

In [ ]:
for i, ID in enumerate(sids_10): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    sub_dir = os.path.join(data_dir, sub)


    dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')
    ## for CBV, create %change    

    if resolution != 'lowres':
        CBV_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbv.nii.gz'))/0.75
        CBV_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbv.nii.gz'))/0.75
    if resolution == 'lowres':
        CBV_task = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbv.nii.gz'))/0.75
        CBV_baseline = masker.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbv.nii.gz'))/0.75        
    CBV_percchange[i, :]  = (CBV_task - CBV_baseline) / CBV_baseline *100

    if resolution != 'lowres':
        CBV_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbv.nii.gz'))/0.75
        CBV_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbv.nii.gz'))/0.75
    if resolution == 'lowres':
        CBV_baseline_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbv.nii.gz'))/0.75
        CBV_task_GM = masker_GM.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbv.nii.gz'))/0.75
    CBV_percchange_GM[i, :]  = (CBV_task_GM - CBV_baseline_GM) / CBV_baseline_GM *100
    
    
## save % change
CBV_percchange_median = np.nanmedian(CBV_percchange, axis=0)

CBV_percchange_median_GM = np.nanmedian(CBV_percchange_GM, axis=0)
   
from matplotlib.colors import ListedColormap
from scipy import stats

colors_lightorange = ListedColormap(["navajowhite"])
colors_lightblue = ListedColormap(["lightskyblue"])
colors=BlueRed

coords = (-30, -15, 0, 15, 30, 45, 60)

## plot %signal change in CBV and CBF vs. BOLD (in color) ##
##############################################################

## group mask ##
###################

### get regression equation and plot line
slope, intercept, r_value, p_value, std_err = stats.linregress(CBV_percchange_median, CBF_percchange_median)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(-30, 40, num=200)



## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(CBV_percchange_median_GM, CBF_percchange_median_GM, c=BOLD_percchange_median_GM, s=2, cmap = colors, vmin=-1, vmax=1)

cbar = plt.colorbar(scatter)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)
# Plot regression line
ax.plot(xseq, intercept + slope * xseq, color="grey", lw=2.5, linestyle='dashed');
ax.text(20, 30, 'r=' + str(round(r_value, 2)), fontsize=fontsize)
    

plt.hlines(0, -30, 40, color='grey', alpha=0.4, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')
#ax.plot([-30, 40], [-30, 40], color='gray', linestyle='--')

ax.set_xlabel('ΔCBV [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
    
ax.set_xlim(-25,35)
ax.set_ylim(-25,35)

fig.savefig(results_dir + '/N10_CBV_CBF'+ '_mask_' +  contrast +  '_'+CMRO2_mode +'.png', dpi=300, bbox_inches='tight')

plt.show()

## BOLD-CBF scatterplot ##
###################

# Create sequence of 100 numbers from 0 to 100 
slope, intercept, r_value, p_value, std_err = stats.linregress(BOLD_percchange_median, CBF_percchange_median)

# Plot regression line
xseq = np.linspace(-30, 40, num=200)

## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(BOLD_percchange_median_GM, CBF_percchange_median_GM, c=CBV_percchange_median_GM, s=2, cmap = colors, vmin=-10, vmax=10)

cbar = plt.colorbar(scatter)
cbar.set_label("% CBV", fontsize=16)
cbar.ax.tick_params(labelsize=14)
# Plot regression line
ax.plot(xseq, intercept + slope * xseq, color="grey", lw=2.5, linestyle='dashed');
ax.text(1.5, 20, 'r=' + str(round(r_value, 2)),  fontsize=fontsize)
        
plt.hlines(0, -30, 40, color='grey', alpha=0.4, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')
ax.set_xticks([-2, -1, 0, 1, 2])

ax.set_xlabel('BOLD [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
    
ax.set_xlim(-2,2.5)
ax.set_ylim(-20,25)

fig.savefig(results_dir + '/N10_BOLD_CBF'+ '_mask_' +  contrast +  '_'+CMRO2_mode +'.png', dpi=300, bbox_inches='tight')

plt.show()


## BOLD-CBV scatterplot ##
###################
# Create sequence of 100 numbers from 0 to 100 
slope, intercept, r_value, p_value, std_err = stats.linregress(BOLD_percchange_median, CBV_percchange_median)

# Plot regression line
xseq = np.linspace(-30, 40, num=200)


## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(BOLD_percchange_median_GM, CBV_percchange_median_GM,c=CBF_percchange_median_GM,  s=2, cmap = colors, vmin=-18, vmax=18)

cbar = plt.colorbar(scatter)
cbar.set_label("% CBF", fontsize=16)   
cbar.ax.tick_params(labelsize=14)

plt.hlines(0, -30, 40, color='grey', alpha=0.4, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')

ax.plot(xseq, intercept + slope * xseq, color="grey", lw=2.5, linestyle='dashed');
ax.text(1.5, 20, 'r=' + str(round(r_value, 2)),  fontsize=fontsize)
ax.set_xticks([-2, -1, 0, 1, 2])

ax.set_xlabel('BOLD [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBV [%]', fontsize=fontsize)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
    
ax.set_xlim(-2,2.5)
ax.set_ylim(-20,25)

fig.savefig(results_dir + '/N10_BOLD_CBV'+ '_mask_' +  contrast +  '_'+CMRO2_mode +'.png', dpi=300, bbox_inches='tight')

plt.show()

### bigger mask ##+


MNI_2mm_brain_mask = derivatives_dir + '/MNI152_T1_2mm_brain_mask.nii.gz'
! fslmaths {MNI_2mm_brain} -bin {MNI_2mm_brain_mask} 
masker_MNI = input_data.NiftiMasker(mask_img = MNI_2mm_brain_mask)
mask_arr = np.array(nib.load(MNI_2mm_brain_mask).dataobj)
mask_len = len (mask_arr[abs(mask_arr)>0])

CBV_percchange = np.zeros((len(sids_10), mask_len))
for i, ID in enumerate(sids_10): #loop over subjects
    sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
    print(sub)

    sub_dir = os.path.join(data_dir, sub)


    dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')
    ## for CBV, create %change    

    if resolution != 'lowres':
        CBV_baseline_GM = masker_MNI.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152_cbv.nii.gz'))/0.75
        CBV_task_GM = masker_MNI.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152_cbv.nii.gz'))/0.75
    if resolution == 'lowres':
        CBV_baseline_GM = masker_MNI.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + baseline + '_space-MNI152-lowres_cbv.nii.gz'))/0.75
        CBV_task_GM = masker_MNI.fit_transform(os.path.join(dir_qmri_deriv, sub + '_task-' + task + '_space-MNI152-lowres_cbv.nii.gz'))/0.75
    CBV_percchange[i, :]  = (CBV_task_GM - CBV_baseline_GM) / CBV_baseline_GM *100

CBV_percchange_median_wholebrain = np.nanmedian(CBV_percchange, axis=0)
CBV_nii_wholebrain = masker_MNI.inverse_transform(CBV_percchange_median_wholebrain)
plotting.plot_img(CBV_nii_wholebrain, bg_img = MNI_2mm_brain, threshold=0, display_mode='z', vmin=-10, vmax=10, cmap='jet', colorbar=True, cut_coords=coords)

## turn into nifti
CBV_nii = masker_GM.inverse_transform(CBV_percchange_median_GM)
plotting.plot_img(CBV_nii, bg_img = MNI_2mm_brain, threshold=0, display_mode='z', vmin=-10, vmax=10, cmap='jet', colorbar=True, cut_coords=coords)

## turn into nifti
BOLD_nii = masker_GM.inverse_transform(BOLD_percchange_median_GM)
plotting.plot_img(BOLD_nii, bg_img = MNI_2mm_brain, threshold=0, display_mode='z', vmin=-1, vmax=1, cmap='jet', colorbar=True, cut_coords=coords)


## Fig. 3C: plot concordant & discordant voxels, separately for positive and negative BOLD

In [ ]:
sns.set_style("whitegrid")
## only look at fully quant CMRO2 and within BOLD PLS region 

coords = (-30, -15, 0, 15, 30, 45, 60)
DMN = os.path.join(derivatives_dir,'Yeo2011_7Networks_MNI152_FreeSurferConformed2mm_MNI_DMN.nii.gz')
colors = BlueRed
colors_bin = ListedColormap(["dodgerblue", "darkorange"])
colors_lightorange = ListedColormap(["navajowhite"])
colors_lightblue = ListedColormap(["paleturquoise"])

## plot only BOLD
BOLD = np.zeros((1, len(BOLD_percchange_median)))
BOLD[0, BOLD_percchange_median>0.001] = 1
BOLD[0, BOLD_percchange_median<-0.001] = -1
BOLD_sum = np.sum(BOLD, axis=0)

BOLD_nifti_bin = masker.inverse_transform(BOLD_sum)
BOLD_nifti = masker.inverse_transform(BOLD_percchange_median)
BOLD_percchange_pos = np.zeros(BOLD_percchange_median.shape)
BOLD_percchange_pos[BOLD_percchange_median>0]=BOLD_percchange_median[BOLD_percchange_median>0]
BOLD_nifti_pos = masker.inverse_transform(BOLD_percchange_pos)

plot=plotting.plot_img(BOLD_nifti_bin, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors_bin, colorbar=True,
                cut_coords = coords,  title=   ' mask: BOLD act / deact')
plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.1)
plot=plotting.plot_img(BOLD_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors, vmin=-1, vmax=1, colorbar=True,
                cut_coords = coords,  title=   ' mask: BOLD act / deact')
plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.1)

##only positive 
plot=plotting.plot_img(BOLD_nifti_pos, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=colors, vmin=-1, vmax=1, colorbar=True,
                cut_coords = coords,  title=   ' mask: BOLD act / deact')
#plot.add_contours(PLS_BOLD_intersect_mask,  colors='mediumseagreen', alpha = 1, linewidths=0.2)
plot.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.2)

##############################################################
fontsize=17 
plt.rcParams['legend.title_fontsize'] = 'x-large'


## group mask ##
###################

## only positive BOLD voxels ##
###############################

BOLD_percchange_median_pos = np.delete(BOLD_percchange_median, np.where(BOLD_percchange_median<=0))
CMRO2_percchange_median_pos = np.delete(CMRO2_percchange_median, np.where(BOLD_percchange_median<=0)) 
CBF_percchange_median_pos = np.delete(CBF_percchange_median, np.where(BOLD_percchange_median<=0)) 

### get regression equation and plot line
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)

#b, a = np.polyfit(CMRO2_percchange_median_pos, CBF_percchange_median_pos, deg=1)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(-40, 40, num=200)


## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))

scatter = ax.scatter(CMRO2_percchange_median_pos, CBF_percchange_median_pos, c=BOLD_percchange_median_pos, s=2, cmap =colors, vmin=-1, vmax=1)
# Plot regression line
#ax.plot(xseq, a + b * xseq, color="grey", lw=2.5, linestyle='dashed');
#ax.text(20, 40, 'b=' + str(round(b, 2)), fontsize=fontsize)

cbar = plt.colorbar(scatter)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)
    
plt.hlines(0, -30, 40, color='black', alpha=0.3, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')

ax.set_xlabel('ΔCMRO2 [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
    
ax.set_xlim(-30,40)
ax.set_ylim(-30,40)
fig.savefig(results_dir + '/CMRO2_CBF'+ '_mask_'  + contrast +  '_pos.png', dpi=300, bbox_inches='tight')

plt.show()


## only negative BOLD voxels ##
###############################
BOLD_percchange_median_neg = np.delete(BOLD_percchange_median, np.where(BOLD_percchange_median>=0))
CMRO2_percchange_median_neg = np.delete(CMRO2_percchange_median, np.where(BOLD_percchange_median>=0)) 
CBF_percchange_median_neg = np.delete(CBF_percchange_median, np.where(BOLD_percchange_median>=0)) 

### get regression equation and plot line
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
#b, a = np.polyfit(CMRO2_percchange_median_neg, CBF_percchange_median_neg, deg=1)

# Create sequence of 100 numbers from 0 to 100 
xseq = np.linspace(-40, 40, num=200)

## plot %signal change in CMRO2 and CBF vs. BOLD (in color)
fig, ax = plt.subplots(1, 1, figsize=(7,5))
scatter = ax.scatter(CMRO2_percchange_median_neg, CBF_percchange_median_neg, c=BOLD_percchange_median_neg, s=2, cmap =colors, vmin=-1, vmax=1)
# Plot regression line
#ax.plot(xseq, a + b * xseq, color="grey", lw=2.5, linestyle='dashed');
#ax.text(20, 40, 'b=' + str(round(b, 2)), fontsize=fontsize)

cbar = plt.colorbar(scatter)
cbar.set_label("% BOLD", fontsize=16)
cbar.ax.tick_params(labelsize=14)

plt.hlines(0, -30, 40, color='black', alpha=0.3, linestyles='dashed')
plt.vlines(0, -30, 40, color='red',alpha=1, linestyles='dashed')

ax.set_xlabel('ΔCMRO2 [%]', fontsize=fontsize)
ax.set_ylabel('ΔCBF [%]', fontsize=fontsize)

ax.set_xlim(-30,40)
ax.set_ylim(-30,40)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)

fig.savefig(results_dir + '/CMRO2_CBF' + '_mask_' + contrast +  '_neg.png', dpi=300, bbox_inches='tight')
    
plt.show()


## pie-chart false increases/decreases ##
########################################


colours_pos= [ 'navajowhite', 'darkviolet'] ## increase, decrease
colours_neg= [ 'paleturquoise', 'darkviolet'] ## decrease, increase

## separate in voxels that show BOLD positive or negative changes
##NBR
CMRO2_neg = CMRO2_percchange_median_neg[BOLD_percchange_median_neg<=0]
CMRO2_neg_pos = len(CMRO2_neg[CMRO2_neg>=0])
CMRO2_neg_neg = len(CMRO2_neg[CMRO2_neg<=0])

##PBR
CMRO2_pos = CMRO2_percchange_median_pos[BOLD_percchange_median_pos>0]
CMRO2_pos_pos = len(CMRO2_pos[CMRO2_pos>=0])
CMRO2_pos_neg = len(CMRO2_pos[CMRO2_pos<=0])

## positive BOLD voxels ##
##NBR
df_neg_CMRO2 = pd.DataFrame({'vox_num':[CMRO2_neg_neg, CMRO2_neg_pos], 'labels':['CMRO2 decrease', 'CMRO2 increase']})
##PBR
df_pos_CMRO2 = pd.DataFrame({'vox_num':[CMRO2_pos_pos, CMRO2_pos_neg], 'labels':['CMRO2 increase', 'CMRO2 decrease']})

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,10))
    
patches, texts, autotexts = ax1.pie(df_pos_CMRO2.vox_num, labels=['',''],autopct = '%0.0f%%', startangle = 79.5, explode = [0.03,0.03],
        textprops = {'fontsize':25}, colors=colours_pos, wedgeprops = {'linewidth': 6, 'alpha':0.8}, center = (0.1,0.1))
autotexts[0].set_color('gray')
autotexts[1].set_color('lightgray')

ax1.set_title('positive BOLD voxels')

ax1.legend( bbox_to_anchor=(0.8, 0), labels = df_pos_CMRO2.labels, fontsize=14, shadow=True)

patches, texts, autotexts = ax2.pie(df_neg_CMRO2.vox_num, labels=['',''],autopct = '%0.0f%%', startangle = 79.5,explode = [0.03,0.03],
        textprops = {'color': 'Lightgray','fontsize':27}, colors=colours_neg, wedgeprops = {'linewidth': 6, 'alpha':0.8}, center = (0.1,0.1))
autotexts[0].set_color('gray')
autotexts[1].set_color('lightgray')
ax2.set_title('negative BOLD voxels')

ax2.legend( bbox_to_anchor=(0.8, 0), labels = df_pos_CMRO2.labels, fontsize=14, shadow=True)

fig.savefig(results_dir + '/Pieplot_'+ '_mask_'  + contrast + '_pos_neg_deviation.png', dpi=300, bbox_inches='tight')

plt.show()



## pie-chart false increases/decreases only in DMN ##
#####################################################
colours_pos= [ 'navajowhite', 'darkviolet'] ## increase, decrease
colours_neg= [ 'paleturquoise', 'darkviolet'] ## decrease, increase

DMN = os.path.join(derivatives_dir,'Yeo2011_7Networks_MNI152_FreeSurferConformed2mm_MNI_DMN.nii.gz')
DMN_GM_arr = masker_GM.fit_transform(DMN)
DMN_GM = masker_GM.inverse_transform(DMN_GM_arr)

DMN_GM_sign_mask = np.squeeze(masker.fit_transform(DMN_GM)) ## same length as BOLD_percchange_median
DMN_GM_bool = DMN_GM_sign_mask.astype(np.bool)

## only DMN voxels
BOLD_percchange = BOLD_percchange_median[DMN_GM_bool]
CMRO2_percchange = CMRO2_percchange_median[DMN_GM_bool]

## separate in voxels that show BOLD positive or negative changes
##NBR
CMRO2_neg = CMRO2_percchange[BOLD_percchange<=0]
CMRO2_neg_pos = len(CMRO2_neg[CMRO2_neg>=0])
CMRO2_neg_neg = len(CMRO2_neg[CMRO2_neg<=0])

##PBR
CMRO2_pos = CMRO2_percchange[BOLD_percchange>0]
CMRO2_pos_pos = len(CMRO2_pos[CMRO2_pos>=0])
CMRO2_pos_neg = len(CMRO2_pos[CMRO2_pos<=0])

## positive BOLD voxels ##
##NBR
df_neg_CMRO2 = pd.DataFrame({'vox_num':[CMRO2_neg_neg, CMRO2_neg_pos ], 'labels':['CMRO2 decrease', 'CMRO2 increase']})
##PBR
df_pos_CMRO2 = pd.DataFrame({'vox_num':[CMRO2_pos_pos, CMRO2_pos_neg], 'labels':['CMRO2 increase', 'CMRO2 decrease']})

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,10))
    
patches, texts, autotexts = ax1.pie(df_pos_CMRO2.vox_num, labels=['',''],autopct = '%0.0f%%', startangle = 79.5, explode = [0.03,0.03],
        textprops = {'fontsize':25}, colors=colours_pos, wedgeprops = {'edgecolor':'k','linewidth': 5}, center = (0.1,0.1))
autotexts[0].set_color('gray')
autotexts[1].set_color('lightgray')

ax1.set_title('positive BOLD voxels within DMN')

ax1.legend( bbox_to_anchor=(0.8, 0), labels = df_pos_CMRO2.labels, fontsize=14, shadow=True)

patches, texts, autotexts = ax2.pie(df_neg_CMRO2.vox_num, labels=['',''],autopct = '%0.0f%%', startangle = 79.5,explode = [0.03,0.03],
        textprops = {'color': 'Lightgray','fontsize':27}, colors=colours_neg, wedgeprops = {'edgecolor':'k','linewidth': 5}, center = (0.1,0.1))
autotexts[0].set_color('gray')
autotexts[1].set_color('lightgray')
ax2.set_title('negative BOLD voxels within DMN')

ax2.legend( bbox_to_anchor=(0.8, 0), labels = df_pos_CMRO2.labels, fontsize=14, shadow=True)

fig.savefig(results_dir + '/Pieplot_'+ '_mask_'  + contrast  + '_pos_neg_devaiation_DMNonly.png', dpi=300, bbox_inches='tight')

plt.show()

print('number of negative BOLD voxels within DMN: ' + str(len(CMRO2_neg)))

print('number of positive BOLD voxels within DMN: ' + str(len(CMRO2_pos)))

## create dataframes with changes, pos/neg BOLD voxels, concordant/discordant

In [ ]:
## plot median changes

## only negative BOLD voxels ##
###############################
BOLD_percchange_median_neg = np.delete(BOLD_percchange_median, np.where(BOLD_percchange_median>=0))
CMRO2_percchange_median_neg = np.delete(CMRO2_percchange_median, np.where(BOLD_percchange_median>=0)) ## where BOLD percchange is negative
CBF_percchange_median_neg = np.delete(CBF_percchange_median, np.where(BOLD_percchange_median>=0)) ## where BOLD percchange is negative

## NBR
BOLD_neg = BOLD_percchange_median_neg[BOLD_percchange_median_neg<0] ## where BOLD percchange is negative
CMRO2_neg = CMRO2_percchange_median_neg[BOLD_percchange_median_neg<0] ## where BOLD percchange is negative
CBF_neg = CBF_percchange_median_neg[BOLD_percchange_median_neg<0] ## where BOLD percchange is negative

## where CMRO2 is also negative: concordant
BOLD_concordant_neg = BOLD_neg[CMRO2_neg<0]
CMRO2_concordant_neg = CMRO2_neg[CMRO2_neg<0]
CBF_concordant_neg = CBF_neg[CMRO2_neg<0]
## where CMRO2 is posiitve: discordant
BOLD_discordant_neg = BOLD_neg[CMRO2_neg>0]
CMRO2_discordant_neg = CMRO2_neg[CMRO2_neg>0]
CBF_discordant_neg = CBF_neg[CMRO2_neg>0]

##PBR
BOLD_percchange_median_pos = np.delete(BOLD_percchange_median, np.where(BOLD_percchange_median<=0))
CMRO2_percchange_median_pos = np.delete(CMRO2_percchange_median, np.where(BOLD_percchange_median<=0)) ## where BOLD percchange is negative
CBF_percchange_median_pos = np.delete(CBF_percchange_median, np.where(BOLD_percchange_median<=0)) ## where BOLD percchange is negative

BOLD_pos = BOLD_percchange_median_pos[BOLD_percchange_median_pos>0]
CMRO2_pos = CMRO2_percchange_median_pos[BOLD_percchange_median_pos>0]
CBF_pos = CBF_percchange_median_pos[BOLD_percchange_median_pos>0] ## where BOLD percchange is negative

## where CMRO2 is also positive: concordant
BOLD_concordant_pos = BOLD_pos[CMRO2_pos>0]
CMRO2_concordant_pos = CMRO2_pos[CMRO2_pos>0]
CBF_concordant_pos = CBF_pos[CMRO2_pos>0]
## where CMRO2 is  negative: discordant
BOLD_discordant_pos = BOLD_pos[CMRO2_pos<0]
CMRO2_discordant_pos = CMRO2_pos[CMRO2_pos<0]
CBF_discordant_pos = CBF_pos[CMRO2_pos<0]


## create dataframes

## only positive BOLD voxels ##
###############################

df_percchange_concordant_pos = pd.DataFrame()
df_percchange_concordant_pos['BOLD'] = BOLD_concordant_pos
df_percchange_concordant_pos['CBF'] = CBF_concordant_pos/10
df_percchange_concordant_pos['CMRO2'] = CMRO2_concordant_pos/10
print('BOLD median concordant pos:' + str(np.nanmedian(df_percchange_concordant_pos['BOLD'])))
print('CBF median concordant pos:' + str(np.nanmedian(df_percchange_concordant_pos['CBF']*10)))
print('CMRO2 median concordant pos:' + str(np.nanmedian(df_percchange_concordant_pos['CMRO2']*10)))
print('n-ratio median concordant pos:' + str(np.nanmedian(df_percchange_concordant_pos['CBF']/df_percchange_concordant_pos['CMRO2'])))

df_percchange_discordant_pos = pd.DataFrame()
df_percchange_discordant_pos['BOLD'] = BOLD_discordant_pos
df_percchange_discordant_pos['CBF'] = CBF_discordant_pos/10
df_percchange_discordant_pos['CMRO2'] = CMRO2_discordant_pos/10
print('BOLD median discordant pos:' + str(np.nanmedian(df_percchange_discordant_pos['BOLD'])))
print('CBF median discordant pos:' + str(np.nanmedian(df_percchange_discordant_pos['CBF']*10)))
print('CMRO" median discordant pos:' + str(np.nanmedian(df_percchange_discordant_pos['CMRO2']*10)))

df_percchange_pos = pd.DataFrame()
for par in ['BOLD', 'CBF', 'CMRO2']:
    df_percchange_pos[par] = np.concatenate((df_percchange_concordant_pos[par], df_percchange_discordant_pos[par]), axis=0)
    df_percchange_pos['ROI'] = np.concatenate((len(df_percchange_concordant_pos[par])*['concordant'], len(df_percchange_discordant_pos[par])*['discordant']), axis=0)
    
df_percchange_pos_melted = pd.melt(df_percchange_pos, id_vars='ROI')

## only neagtive BOLD voxels ##
###############################

df_percchange_concordant_neg = pd.DataFrame()
df_percchange_concordant_neg['BOLD'] = BOLD_concordant_neg
df_percchange_concordant_neg['CBF'] = CBF_concordant_neg/10
df_percchange_concordant_neg['CMRO2'] = CMRO2_concordant_neg/10
print('BOLD median concordant neg:' + str(np.nanmedian(df_percchange_concordant_neg['BOLD'])))
print('CBF median concordant neg:' + str(np.nanmedian(df_percchange_concordant_neg['CBF']*10)))
print('CMRO2 median concordant neg:' + str(np.nanmedian(df_percchange_concordant_neg['CMRO2']*10)))
print('n-ratio median concordant neg:' + str(np.nanmedian(df_percchange_concordant_neg['CBF']/df_percchange_concordant_neg['CMRO2'])))

df_percchange_discordant_neg = pd.DataFrame()
df_percchange_discordant_neg['BOLD'] = BOLD_discordant_neg
df_percchange_discordant_neg['CBF'] = CBF_discordant_neg/10
df_percchange_discordant_neg['CMRO2'] = CMRO2_discordant_neg/10
print('BOLD median discordant neg:' + str(np.nanmedian(df_percchange_discordant_neg['BOLD'])))
print('CBF median discordant neg:' + str(np.nanmedian(df_percchange_discordant_neg['CBF']*10)))
print('CMRO2 median discordant neg:' + str(np.nanmedian(df_percchange_discordant_neg['CMRO2']*10)))

df_percchange_neg = pd.DataFrame()
for par in ['BOLD', 'CBF', 'CMRO2']:
    df_percchange_neg[par] = np.concatenate((df_percchange_concordant_neg[par], df_percchange_discordant_neg[par]), axis=0)
    df_percchange_neg['ROI'] = np.concatenate((len(df_percchange_concordant_neg[par])*['concordant'], len(df_percchange_discordant_neg[par])*['discordant']), axis=0)
    
df_percchange_neg_melted = pd.melt(df_percchange_neg, id_vars='ROI')



## Fig. 3C: BOLD quartiles: discordant/concordant voxels

In [ ]:
## add barplot: percentile BOLD false pos./neg##
################################################
perc25 = np.percentile(BOLD_percchange_median_pos, 25)
perc50 = np.percentile(BOLD_percchange_median_pos, 50)
perc75 = np.percentile(BOLD_percchange_median_pos, 75)


false_pos = BOLD_percchange_median_pos[CMRO2_percchange_median_pos<0]
correct_pos = BOLD_percchange_median_pos[CMRO2_percchange_median_pos>0]

BOLDpos = np.zeros((2,4))
BOLDpos[0,0] = len(correct_pos[correct_pos<perc25])
BOLDpos[0,1] = len(np.where(correct_pos[correct_pos<perc50]>perc25)[0])
BOLDpos[0,2] = len(np.where(correct_pos[correct_pos<perc75]>perc50)[0])
BOLDpos[0,3] = len(correct_pos[correct_pos>perc75])
BOLDpos[1,0] = len(false_pos[false_pos<perc25])
BOLDpos[1,1] = len(np.where(false_pos[false_pos<perc50]>perc25)[0])
BOLDpos[1,2] = len(np.where(false_pos[false_pos<perc75]>perc50)[0])
BOLDpos[1,3] =len(false_pos[false_pos>perc75])


fig, ax = plt.subplots(1, figsize=(3,2))
labels=('q1', 'q2', 'q3', 'q4')
#labels=('0 - '+str(round(perc25,2))+'%', str(round(perc25,2))+' - '+str(round(perc50,2))+'%', str(round(perc50,2))+' - '+str(round(perc75,2))+'%', str(round(perc75,2))+' - '+str(round(np.max(BOLD_percchange_median_pos),2))+'%')
#labels=('0-25%', '25-50%', '50-75%', '75-100%')
ax.bar(labels, BOLDpos[1,:], 0.8, label='false pos.', color='mediumorchid')
ax.bar(labels, BOLDpos[0,:], 0.8, bottom=BOLDpos[1,:],  label='correct pos.', color='navajowhite')
ax.set_ylabel('N° of voxels', fontsize=fontsize)
#plt.xticks(rotation=30, ha='right')
ax.set_xlabel('BOLD quartiles', fontsize=fontsize)
        
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)

for pos in ['right', 'top']:
        ax.spines[pos].set_visible(False)
            
#ax.legend()
plt.show()

fig.savefig(results_dir + '/Barplot25%bins_mask_'  + contrast + '_false_pos.png', dpi=300, bbox_inches='tight')


perc25 = np.percentile(BOLD_percchange_median_neg, 25)
perc50 = np.percentile(BOLD_percchange_median_neg, 50)
perc75 = np.percentile(BOLD_percchange_median_neg, 75)

false_neg = BOLD_percchange_median_neg[CMRO2_percchange_median_neg>0]
correct_neg = BOLD_percchange_median_neg[CMRO2_percchange_median_neg<0]


BOLDneg = np.zeros((2,4))
BOLDneg[0,0] = len(correct_neg[correct_neg>perc75])
BOLDneg[0,1] = len(np.where(correct_neg[correct_neg<perc75]>perc50)[0])
BOLDneg[0,2] = len(np.where(correct_neg[correct_neg<perc50]>perc25)[0])
BOLDneg[0,3] = len(correct_neg[correct_neg<perc25])
BOLDneg[1,0] = len(false_neg[false_neg>perc75])
BOLDneg[1,1] = len(np.where(false_neg[false_neg<perc75]>perc50)[0])
BOLDneg[1,2] = len(np.where(false_neg[false_neg<perc50]>perc25)[0])
BOLDneg[1,3] = len(false_neg[false_neg<perc25])



fig, ax = plt.subplots(1, figsize=(3,2))
labels=('q1', 'q2', 'q3', 'q4')
#labels=('0 - '+str(round(perc75,2))+'%', str(round(perc75,2))+' - '+str(round(perc50,2))+'%', str(round(perc50,2))+' - '+str(round(perc25,2))+'%', str(round(perc25,2))+' - '+str(round(np.min(BOLD_percchange_median_neg),2))+'%')
#labels=('0-25%', '25-50%', '50-75%', '75-100%')
ax.bar(labels, BOLDneg[1,:], 0.8,  label='false neg.', color='mediumorchid')
ax.bar(labels, BOLDneg[0,:], 0.8, bottom=BOLDneg[1,:], label='correct neg.', color='paleturquoise')
ax.set_xlabel('BOLD quartiles', fontsize=fontsize)
#ax.legend()
ax.set_ylabel('N° of voxels', fontsize=fontsize)
#plt.xticks(rotation=30, ha='right')
        
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)

for pos in ['right', 'top']:
        ax.spines[pos].set_visible(False)

plt.show()
fig.savefig(results_dir + '/Barplot25%bins_mask_'  + contrast + '_false_neg.png', dpi=300, bbox_inches='tight')


## Fig. 3C: brain plots: discordant and concordant voxels

### NOT USED: whole-brain

In [ ]:
from matplotlib.colors import ListedColormap
DMN = os.path.join(derivatives_dir,'Yeo2011_7Networks_MNI152_FreeSurferConformed2mm_MNI_DMN.nii.gz')

cmap_neg = ListedColormap(["darkviolet", "darkviolet", "paleturquoise", "paleturquoise"  ])
cmap_pos = ListedColormap(["darkviolet", "darkviolet", "navajowhite","navajowhite"])

neg_BOLD_idx = np.where(BOLD_percchange_median_GM<0)
pos_BOLD_idx = np.where(BOLD_percchange_median_GM>0)

neg_CMRO2_idx = np.where(CMRO2_percchange_median_GM<0)
pos_CMRO2_idx = np.where(CMRO2_percchange_median_GM>0)

neg_BOLD_pos_CMRO2 = np.intersect1d(neg_BOLD_idx[0], pos_CMRO2_idx[0])
neg_BOLD_neg_CMRO2 = np.intersect1d(neg_BOLD_idx[0], neg_CMRO2_idx[0])
pos_BOLD_neg_CMRO2 = np.intersect1d(pos_BOLD_idx[0], neg_CMRO2_idx[0])
pos_BOLD_pos_CMRO2 = np.intersect1d(pos_BOLD_idx[0], pos_CMRO2_idx[0])

##create niftis 
neg_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
neg_BOLD_pos_CMRO2_nii[neg_BOLD_pos_CMRO2] = 1
neg_BOLD_pos_CMRO2_nifti = masker_GM.inverse_transform(neg_BOLD_pos_CMRO2_nii)
nib.save(neg_BOLD_pos_CMRO2_nifti, os.path.join(results_dir, 'negBOLD_posCMRO2_' + contrast +'_' + whole_brain_masking + '.nii.gz'))
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='discording voxels negBOLD')

neg_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
neg_BOLD_neg_CMRO2_nii[neg_BOLD_neg_CMRO2] = 1
neg_BOLD_neg_CMRO2_nifti = masker_GM.inverse_transform(neg_BOLD_neg_CMRO2_nii)
nib.save(neg_BOLD_neg_CMRO2_nifti, os.path.join(results_dir, 'negBOLD_negCMRO2_' + contrast +'_' + whole_brain_masking  +'.nii.gz'))
disp=plotting.plot_img(neg_BOLD_neg_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='concording voxels negBOLD')

pos_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
pos_BOLD_neg_CMRO2_nii[pos_BOLD_neg_CMRO2] = 1
pos_BOLD_neg_CMRO2_nifti = masker_GM.inverse_transform(pos_BOLD_neg_CMRO2_nii)
nib.save(pos_BOLD_neg_CMRO2_nifti, os.path.join(results_dir,  'posBOLD_negCMRO2_' + contrast + '_' + whole_brain_masking+ '.nii.gz'))
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='discording voxels posBOLD')

pos_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
pos_BOLD_pos_CMRO2_nii[pos_BOLD_pos_CMRO2] = 1
pos_BOLD_pos_CMRO2_nifti = masker_GM.inverse_transform(pos_BOLD_pos_CMRO2_nii)
nib.save(pos_BOLD_pos_CMRO2_nifti, os.path.join(results_dir, 'posBOLD_posCMRO2_' + contrast +'_' + whole_brain_masking + '.nii.gz'))
disp=plotting.plot_img(pos_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='concording voxels posBOLD')

##create nifti :negative BOLD
neg_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
neg_BOLD_pos_CMRO2_nii[neg_BOLD_pos_CMRO2] = 2 ## discordant
neg_BOLD_pos_CMRO2_nii[neg_BOLD_neg_CMRO2] = 4 ## concordant
neg_BOLD_pos_CMRO2_nifti = masker_GM.inverse_transform(neg_BOLD_pos_CMRO2_nii)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, colorbar=True)
if task == 'calc':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_negBOLD_' + contrast +  '.png')
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_neg, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5, colorbar=True)
if task == 'calc':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_neg, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_negBOLD_' + contrast +  '_sag.png')



##create nifti :positive BOLD

pos_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median_GM))
pos_BOLD_neg_CMRO2_nii[pos_BOLD_neg_CMRO2] = 2 ## discordant
pos_BOLD_neg_CMRO2_nii[pos_BOLD_pos_CMRO2] = 4 ## concordant
pos_BOLD_neg_CMRO2_nii_nifti = masker_GM.inverse_transform(pos_BOLD_neg_CMRO2_nii)

disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_pos, 
                cut_coords = coords, vmin = 1, vmax =5, colorbar=True)
if task == 'mem':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_pos, 
                cut_coords = coords, vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_posBOLD_'+ contrast +  '.png')
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_pos, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5, colorbar=True)
if task == 'mem':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_pos, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_posBOLD_' + contrast +  '_sag.png')



### only within group mask

In [ ]:
from matplotlib.colors import ListedColormap
DMN = os.path.join(derivatives_dir,'Yeo2011_7Networks_MNI152_FreeSurferConformed2mm_MNI_DMN.nii.gz')

cmap_neg = ListedColormap(["darkviolet", "darkviolet", "paleturquoise", "paleturquoise"  ])
cmap_pos = ListedColormap(["darkviolet", "darkviolet", "navajowhite","navajowhite"])

neg_BOLD_idx = np.where(BOLD_percchange_median<0)
pos_BOLD_idx = np.where(BOLD_percchange_median>0)

neg_CMRO2_idx = np.where(CMRO2_percchange_median<0)
pos_CMRO2_idx = np.where(CMRO2_percchange_median>0)

neg_BOLD_pos_CMRO2 = np.intersect1d(neg_BOLD_idx[0], pos_CMRO2_idx[0])
neg_BOLD_neg_CMRO2 = np.intersect1d(neg_BOLD_idx[0], neg_CMRO2_idx[0])
pos_BOLD_neg_CMRO2 = np.intersect1d(pos_BOLD_idx[0], neg_CMRO2_idx[0])
pos_BOLD_pos_CMRO2 = np.intersect1d(pos_BOLD_idx[0], pos_CMRO2_idx[0])

##create niftis 
neg_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
neg_BOLD_pos_CMRO2_nii[neg_BOLD_pos_CMRO2] = 1
neg_BOLD_pos_CMRO2_nifti = masker.inverse_transform(neg_BOLD_pos_CMRO2_nii)
nib.save(neg_BOLD_pos_CMRO2_nifti, os.path.join(results_dir, 'negBOLD_posCMRO2_' + contrast +'_' + whole_brain_masking + '.nii.gz'))
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='discording voxels negBOLD')

neg_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
neg_BOLD_neg_CMRO2_nii[neg_BOLD_neg_CMRO2] = 1
neg_BOLD_neg_CMRO2_nifti = masker.inverse_transform(neg_BOLD_neg_CMRO2_nii)
nib.save(neg_BOLD_neg_CMRO2_nifti, os.path.join(results_dir, 'negBOLD_negCMRO2_' + contrast +'_' + whole_brain_masking  +'.nii.gz'))
disp=plotting.plot_img(neg_BOLD_neg_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='concording voxels negBOLD')

pos_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
pos_BOLD_neg_CMRO2_nii[pos_BOLD_neg_CMRO2] = 1
pos_BOLD_neg_CMRO2_nifti = masker.inverse_transform(pos_BOLD_neg_CMRO2_nii)
nib.save(pos_BOLD_neg_CMRO2_nifti, os.path.join(results_dir,  'posBOLD_negCMRO2_' + contrast + '_' + whole_brain_masking+ '.nii.gz'))
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='discording voxels posBOLD')

pos_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
pos_BOLD_pos_CMRO2_nii[pos_BOLD_pos_CMRO2] = 1
pos_BOLD_pos_CMRO2_nifti = masker.inverse_transform(pos_BOLD_pos_CMRO2_nii)
nib.save(pos_BOLD_pos_CMRO2_nifti, os.path.join(results_dir, 'posBOLD_posCMRO2_' + contrast +'_' + whole_brain_masking + '.nii.gz'))
disp=plotting.plot_img(pos_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, title='concording voxels posBOLD')

##create nifti :negative BOLD
neg_BOLD_pos_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
neg_BOLD_pos_CMRO2_nii[neg_BOLD_pos_CMRO2] = 2 ## discordant
neg_BOLD_pos_CMRO2_nii[neg_BOLD_neg_CMRO2] = 4 ## concordant
neg_BOLD_pos_CMRO2_nifti = masker.inverse_transform(neg_BOLD_pos_CMRO2_nii)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5, colorbar=True)
if task == 'calc':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_neg, 
                cut_coords = coords, vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_negBOLD_' + contrast +  '.png')
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_neg, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5, colorbar=True)
if task == 'calc':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(neg_BOLD_pos_CMRO2_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_neg, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_negBOLD_' + contrast +  '_sag.png')



##create nifti :positive BOLD

pos_BOLD_neg_CMRO2_nii = np.zeros(len(BOLD_percchange_median))
pos_BOLD_neg_CMRO2_nii[pos_BOLD_neg_CMRO2] = 2 ## discordant
pos_BOLD_neg_CMRO2_nii[pos_BOLD_pos_CMRO2] = 4 ## concordant
pos_BOLD_neg_CMRO2_nii_nifti = masker.inverse_transform(pos_BOLD_neg_CMRO2_nii)

disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_pos, 
                cut_coords = coords, vmin = 1, vmax =5, colorbar=True)
if task == 'mem':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='z', cmap=cmap_pos, 
                cut_coords = coords, vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_posBOLD_'+ contrast +  '.png')
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_pos, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5, colorbar=True)
if task == 'mem':
    disp.add_contours(DMN,  colors='black', alpha = 1, linewidths=0.3)
disp=plotting.plot_img(pos_BOLD_neg_CMRO2_nii_nifti, bg_img=MNI_2mm_brain, threshold = 0, display_mode='x', cmap=cmap_pos, 
                cut_coords = [-40,-4,4, 40], vmin = 1, vmax = 5,
                output_file=results_dir + '/CMRO2_deviation_posBOLD_' + contrast +  '_sag.png')



## Fig. 3D: all changes in one plot, boxplots; only negative & positive BOLD voxels, concordant/discordant

In [ ]:
## pos
colormap = (["navajowhite", "darkviolet" ])
fig, ax = plt.subplots(1, figsize= (6,5))

sns.boxplot(x='variable', y='value', hue='ROI', data=df_percchange_pos_melted, ax=ax, palette=colormap, saturation=0.9,
                   boxprops={"edgecolor": (.6, .6, .6, .5)}, flierprops={"marker": "x"},
                   medianprops={"color": "black"})

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)
    #ax.set_title('coupling CBF/BOLD, CMRO2/BOLD')

if task[0] != 'mem':
    color = 'darkorange'
if task[0] == 'mem':
    color = 'darkorange'
ax.set_ylim(-1.8, 2.1)
ax.set_yticks([-1.5, -1,  -0.5, 0,  0.5,  1, 1.5, 2])
#if baseline == 'rest':
#    ax.set_ylim(-2, 2)
#    ax.set_yticks([-1, -0.5, 0,  0.5, 1.5,  2])
#ax.set_ylim(-0.2, 1.25)
#ax.set_yticks([0, 0.25, 0.5, 0.75, 1, 1.25])
## set line at y=0
ax.hlines(0, -0.7, 2.7, colors='red', linestyles='solid')
ax.tick_params(axis='y', labelcolor=color)
ax.set_ylabel('ΔBOLD [%]', fontsize = fontsize, color=color)  # we already handled the x-label with ax1
ax.set_xticklabels(['BOLD', 'CBF', 'CMRO2'])  # we already handled the x-label with ax1
[t.set_color(i) for (i,t) in zip([color,'black','black'],ax.xaxis.get_ticklabels())]
#ax.legend(fontsize=fontsize-5,loc="upper left")
ax.legend_.remove()

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

color = 'black'
#ax2.plotsns.barplot(x="ROI", y="percchange", hue="parameter", data=df_percchange_median, estimator=np.median, ci=95, capsize=.05, n_boot=2000, errwidth=1,
#                 saturation=0.9, alpha=0.8)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-18, 21)
ax2.set_yticks([-15, -10, -5, 0, 5, 10, 15, 20])
#if baseline == 'rest':
#    ax2.set_ylim(-20, 21)
#    ax2.set_yticks([-20, -15, -10, -5, 0,  5, 10, 15, 20])

#ax2.set_ylim(-2, 12.5)
#ax2.set_yticks([0, 2.5, 5, 7.5, 10, 12.5])

ax2.set_ylabel('ΔCBF[%], ΔCMRO2[%]', color=color, fontsize = fontsize)  # we already handled the x-label with ax1

#set x-tick and y-ticke label size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
    label.set_fontsize(fontsize)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
fig.show()

fig.savefig(results_dir + '/N' + N_subj + '_BOLD_' +resolution + par + '_inline_oppos_BOLD_CMRO2_CBF_pos.png', dpi=300, bbox_inches='tight')

In [ ]:
## neg
colormap = (["paleturquoise", "darkviolet" ])
fig, ax = plt.subplots(1, figsize= (6,5))

sns.boxplot(x='variable', y='value', hue='ROI', data=df_percchange_neg_melted, ax=ax, palette=colormap, saturation=0.9,
                   boxprops={"edgecolor": (.6, .6, .6, .5)}, flierprops={"marker": "x"},
                   medianprops={"color": "black"})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)
    #ax.set_title('coupling CBF/BOLD, CMRO2/BOLD')

if task[0] != 'mem':
    color = 'dodgerblue'
if task[0] == 'mem':
    color = 'dodgerblue'
ax.set_ylim(-1.8, 2.1)
ax.set_yticks([-1.5, -1,  -0.5, 0,  0.5,  1, 1.5, 2])
#if baseline == 'rest':
#    ax.set_ylim(-2, 3)
#    ax.set_yticks([-1, -0.5, 0,  0.5, 1.5,  2])
#ax.set_ylim(-0.2, 1.25)
#ax.set_yticks([0, 0.25, 0.5, 0.75, 1, 1.25])
## set line at y=0
ax.hlines(0, -0.7, 2.7, colors='red', linestyles='solid')
ax.tick_params(axis='y', labelcolor=color)
ax.set_ylabel('ΔBOLD [%]', fontsize = fontsize, color=color)  # we already handled the x-label with ax1
ax.set_xticklabels(['BOLD', 'CBF', 'CMRO2'])  # we already handled the x-label with ax1
[t.set_color(i) for (i,t) in zip([color,'black','black'],ax.xaxis.get_ticklabels())]
#ax.legend(fontsize=fontsize-5,loc="upper left")
ax.legend_.remove()

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

color = 'black'
#ax2.plotsns.barplot(x="ROI", y="percchange", hue="parameter", data=df_percchange_median, estimator=np.median, ci=95, capsize=.05, n_boot=2000, errwidth=1,
#                 saturation=0.9, alpha=0.8)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-18, 21)
ax2.set_yticks([-15, -10, -5, 0, 5, 10, 15, 20])
#if baseline == 'rest':
#    ax2.set_ylim(-20, 20)
#    ax2.set_yticks([-20, -15, -10, -5, 0,  5, 10, 15, 20])

#ax2.set_ylim(-2, 12.5)
#ax2.set_yticks([0, 2.5, 5, 7.5, 10, 12.5])

ax2.set_ylabel('ΔCBF[%], ΔCMRO2[%]', color=color, fontsize = fontsize)  # we already handled the x-label with ax1

#set x-tick and y-ticke label size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
    label.set_fontsize(fontsize)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
fig.show()

fig.savefig(results_dir + '/N' + N_subj + '_BOLD_' +resolution + '_' + par + '_inline_oppos_BOLD_CMRO2_CBF_neg.png', dpi=300, bbox_inches='tight')        